In [4]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

def abs_dist(rho, sigma):
    """A function to compute the absolute value |rho - sigma|."""
    polar = scipy.linalg.polar(rho - sigma)
    return polar[1]

def word_dist(word):
    """A function which counts the non-identity operators in a Pauli word"""
    return sum(word[i] != "I" for i in range(len(word)))


# Produce the Pauli density for a given Pauli word and apply noise

def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """


    # Put your code here #
    n = len(word)
    paulis = {
        "I": np.eye(2),
        "X": np.array([[0, 1], [1, 0]]),
        "Y": np.array([[0, -1j], [1j, 0]]),
        "Z": np.array([[1, 0], [0, -1]])
    }
    
    combined_pauli = np.kron(paulis[word[0]], paulis[word[1]])
    for i in range(2, n):
        combined_pauli = np.kron(combined_pauli, paulis[word[i]])
    combined_pauli = (np.eye(2**n)+combined_pauli)/2**n
    if np.trace(combined_pauli)!=1:
        print(pauli_tensor[i],word[i])
    
    dev = qml.device("default.mixed", wires=n)
    @qml.qnode(dev)
    def circuit():
        qml.QubitDensityMatrix(combined_pauli, wires = list(range(n)))
        for i in range(n):
            qml.DepolarizingChannel(lmbda, wires = i)
            
        return qml.state()
    return(circuit())


# Compute the trace distance from a noisy Pauli density to the maximally mixed density

def maxmix_trace_dist(word, lmbda):
    """
       A function compute the trace distance between a noisy density matrix, specified
       by a Pauli word, and the maximally mixed matrix.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The trace distance between two matrices encoding Pauli words.
    """


    # Put your code here #
    rho_P = noisy_Pauli_density(word,lmbda)
    n=len(word)
    rho_0 = np.eye(2**n)/(2**n)
    return 0.5*np.trace(abs_dist(rho_P, rho_0))


def bound_verifier(word, lmbda):
    """
       A simple check function which verifies the trace distance from a noisy Pauli density
       to the maximally mixed matrix is bounded by (1 - lambda)^|P|.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The difference between (1 - lambda)^|P| and T(rho_P(lambda), rho_0).
    """


    # Put your code here #
    RHS = maxmix_trace_dist(word,lmbda)
    LHS = (1-lmbda)**word_dist(word)
    return LHS-RHS


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    word, lmbda = json.loads(test_case_input)
    output = np.real(bound_verifier(word, lmbda))

    return str(output)


def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your trace distance isn't quite right!"


test_cases = [['["XXI", 0.7]', '0.0877777777777777'], ['["XXIZ", 0.1]', '0.4035185185185055'], ['["YIZ", 0.3]', '0.30999999999999284'], ['["ZZZZZZZXXX", 0.1]', '0.22914458207245006']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '["XXI", 0.7]'...
Correct!
Running test case 1 with input '["XXIZ", 0.1]'...
Correct!
Running test case 2 with input '["YIZ", 0.3]'...
Correct!
Running test case 3 with input '["ZZZZZZZXXX", 0.1]'...
Correct!


Whenever Zenda and Reece push the button on a box and output a state in order to test it, it goes into a noisy circuit, where each qubit is subject to depolarizing noise,  If  is a single-qubit density matrix,  is defined by

and with probability , the state is deleted and replaced with something random. Zenda and Reece suspect that noisy is making the states coming out of the box very hard to distinguish from random, and would like some way to test just how badly blurred they are.

To explore this, we first note that any density matrix on  qubits can be written as a linear combination of a special set of "Pauli" density matrices. These have the form

where  is a tensor product of  single-qubit Pauli operators, called a Pauli word. We'll let  label the result of applying a layer of depolarizing noise to the Pauli density 

If adding noise makes a Pauli density matrix look random, a combination of Pauli densities — in other words, any matrix! — will look random. Here, "looks random" means "the expectation of any measurement is similar to the maximally mixed density matrix ". Remarkably, we can capture all expectations at once using something called trace distance between density matrices. This is defined as

where  for a generic matrix  (to calculate  you will be provided with the function abs_dist). For any (projective) measurement , the trace distance between two density matrices  and  bounds the difference in expectations:

If the trace distance is small, the two states are hard to tell apart with any measurement.

Zenda and Reece suspect that the noise in their circuitry is blurring the states and making them hard to distinguish. Your goal is to write a function which verifies the bound

by computing the difference between the right-hand side and left-hand side, where  is the number of non-identity operators in the Pauli word  You should find this is always positive! Since a Pauli density matrix gets exponentially blurry, and all states can be built from these Pauli densities, most states will be exponentially hard to distinguish.

